In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

In [ ]:
from sklearn.datasets import load_boston
boston = load_boston()

## boston es un diccionario

In [ ]:
print(boston.keys())

## Cantidad de filas y columnas

In [ ]:
boston.data.shape

## Nombre de los features

In [ ]:
print(boston.feature_names)

## Descripción del dataset

In [ ]:
print(boston.DESCR)

## Convertimos a un dataframe de pandas

In [ ]:
import pandas as pd

data = pd.DataFrame(boston.data)
data.columns = boston.feature_names

## Primeras 5 rows

In [ ]:
data.head()

## Agregamos el target al dataframe

In [ ]:
data['PRICE'] = boston.target

## Información sobre los datos en el dataframe

In [ ]:
data.info()

## Información estadística de los features

In [ ]:
data.describe()

## Separamos la variable a predecir

In [ ]:
X, y = data.iloc[:,:-1],data.iloc[:,-1]

## Convertimos los datos a DMatrix

In [ ]:
data_dmatrix = xgb.DMatrix(data=X,label=y)

## Hiper-parámetros

* **learning_rate:** tasa de aprendizaje
* **max_depth:** máxima profundidad de cada árbol
* **subsample:** porcentaje de muestras usadas para cada árbol (valor muy bajo, posible underfitting)
* **colsample_bytree:** porcentaje de features usadas para cada árbol (valores muy alto, posible overfitting)
* **n_estimators:** cantidad de árboles a construir.
* **objective:** función de error a utilizar (algunas: reg:linear para regresión, reg:logistic o binary:logistic para clasificación)

Parámetros de regularización:

* **gamma:** umbral para hacer split basado en la reducción de error de hacer el nuevo split.
* **alpha:** regularización para los pesos de las hojas. Un valor más alto genera una mayor regularización.
* **lambda:** similar alpha pero para la sintonia fina.

## Creamos set de entrenamiento y test

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

## Instanciamos el regresor de XGBoost

In [ ]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

## Entrenamos

In [ ]:
xg_reg.fit(X_train,y_train)

## Predecimos

In [ ]:
preds = xg_reg.predict(X_test)

## Calculamos el error en las predicciones

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

## K-fold cross validation

Se usa el método **cv**

Hiper-parámetros:

* **nfolds:** K del k-fold
* **num_boost_round:** cantidad de árboles a contruir (n_estimators)
* **metrics:** la métrica de evaluación a utilizar
* **as_pandas:** si los resultados lo devuelve en un DataFrame de pandas
* **early_stopping_rounds:** terminar antes si la métrica no mejora luego de una cantidad de pasadas
* **seed:** semilla para poder reproducir los resultados


In [ ]:
params = {"objective":"reg:linear",'colsample_bytree': 0.3,
          'learning_rate': 0.1, 'max_depth': 5, 'alpha': 10}

cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,
                    num_boost_round=50, early_stopping_rounds=10,
                    metrics="rmse", as_pandas=True, seed=123)

## Información de los resultados

In [ ]:
cv_results.head()

## Error final

In [ ]:
print((cv_results["test-rmse-mean"]).tail(1))

## Visualizando los árboles

In [ ]:
xg_reg = xgb.train(params=params, dtrain=data_dmatrix, num_boost_round=10)

In [ ]:
import matplotlib.pyplot as plt

xgb.plot_tree(xg_reg,num_trees=0)
plt.rcParams['figure.figsize'] = [100, 100]
plt.show()

## Importancia de los features (cuantas veces aparecen en los árboles)

In [ ]:
xgb.plot_importance(xg_reg)
plt.rcParams['figure.figsize'] = [10, 10]
plt.show()